# Setting up a Ray cluster with SmartSim

## 1. Start the cluster
We set up a SmartSim experiment, which will handle the launch of the Ray cluster.

First we import the relevant modules.

In [1]:
import numpy as np
import time
import argparse
import os

from ray.tune.progress_reporter import JupyterNotebookReporter
import ray
from ray import tune
import ray.util

from smartsim import Experiment
from smartsim.ray import RayCluster

NUM_WORKERS = 3
alloc=None
#alloc=slurm.get_allocation(nodes=1+NUM_WORKERS, time="12:00:00", options={"ntasks": str(1+NUM_WORKERS), "partition": "spider", "C": "V100"})

In [2]:
exp = Experiment("ray-cluster", launcher='pbs')
cluster = RayCluster(name="ray-cluster", run_args={}, path='',
                     launcher='pbs', workers=NUM_WORKERS, alloc=alloc, batch=True, ray_num_cpus=18)

if cluster.batch:
    cluster.head_model.batch_settings._preamble += ["source ~/.bashrc", "conda activate smartsim"]
    if NUM_WORKERS:
        cluster.worker_model.batch_settings._preamble += ["source ~/.bashrc", "conda activate smartsim"]

exp.generate(cluster, overwrite=True)

Executable: /lus/scratch/arigazzi/anaconda3/envs/smartsim/bin/python
Executable arguments: ['/lus/scratch/arigazzi/smartsim-dev/SmartSim/smartsim/ray/raystarter.py', '--num-cpus=18', '--port=6780', '--redis-password=cb2af36a-7d31-492b-a822-f895a9d426df']
Run Command: aprun
Run arguments: {'sync-output': None}
07:49:02 swan SmartSim[9468] INFO Working in previously created experiment


In [3]:
exp.start(cluster, block=False, summary=False)

07:49:27 swan SmartSim[9468] DEBUG Added step command to batch for head-CBLHL6YRZQ0B
07:49:27 swan SmartSim[9468] DEBUG Gleaned batch job id: 37800.sdb for head-CBLHL6YRZ3P9
07:49:27 swan SmartSim[9468] DEBUG Launching head
07:49:41 swan SmartSim[9468] DEBUG WLM Ray head node acquisition unsupported
07:49:41 swan SmartSim[9468] DEBUG Added step command to batch for workers-CBLHLDHBME4H
07:49:41 swan SmartSim[9468] DEBUG Gleaned batch job id: 37801.sdb for workers-CBLHLDHBLATC
07:49:41 swan SmartSim[9468] DEBUG Launching workers
07:49:41 swan SmartSim[9468] DEBUG WLM Ray worker node acquisition unsupported
07:49:41 swan SmartSim[9468] INFO Ray cluster launched.
07:49:41 swan SmartSim[9468] DEBUG Starting Job Manager


## 2. Start the ray driver script

In [4]:
ray.util.connect(cluster.head_model.address +":10001")

tune.run(
    "PPO",
    stop={"episode_reward_max": 200},
    config={
        "framework": "torch",
        "env": "CartPole-v0",
        "num_gpus": 0,
        "lr": tune.grid_search(np.linspace (0.001, 0.01, 50).tolist()),
        "log_level": "ERROR",
        "num_cpus_per_worker": 1,
    },
    local_dir="/lus/scratch/arigazzi/ray_local/",
    verbose=1,
    fail_fast=True,
    log_to_file=True,
)

(pid=74649) == Status ==
(pid=74649) Memory usage on this node: 5.0/125.8 GiB
(pid=74649) Using FIFO scheduling algorithm.
(pid=74649) Resources requested: 3.0/72 CPUs, 0/0 GPUs, 0.0/339.53 GiB heap, 0.0/149.51 GiB objects
(pid=74649) Result logdir: /lus/scratch/arigazzi/ray_local/PPO
(pid=74649) Number of trials: 50/50 (49 PENDING, 1 RUNNING)
(pid=74649) 
(pid=74649) 
(pid=74649) == Status ==
(pid=74649) Memory usage on this node: 5.4/125.8 GiB
(pid=74649) Using FIFO scheduling algorithm.
(pid=74649) Resources requested: 18.0/72 CPUs, 0/0 GPUs, 0.0/339.53 GiB heap, 0.0/149.51 GiB objects
(pid=74649) Result logdir: /lus/scratch/arigazzi/ray_local/PPO
(pid=74649) Number of trials: 50/50 (44 PENDING, 6 RUNNING)
(pid=74649) 
(pid=74649) 


(pid=74676) 2021-05-24 07:53:58,592	INFO trainer.py:694 -- Current log_level is ERROR. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=74672) 2021-05-24 07:53:58,821	INFO trainer.py:694 -- Current log_level is ERROR. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=74674) 2021-05-24 07:53:58,835	INFO trainer.py:694 -- Current log_level is ERROR. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=74670) 2021-05-24 07:53:59,145	INFO trainer.py:694 -- Current log_level is ERROR. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=74667) 2021-05-24 07:53:59,225	INFO trainer.py:694 -- Current log_level is ERROR. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=74665) 2021-05-24 07:53:59,355	INFO trainer.py:694 -- Current log_level is ERROR. For more information, set 'log_level': 'INFO' / 'DEBU

(pid=74649) == Status ==
(pid=74649) Memory usage on this node: 6.6/125.8 GiB
(pid=74649) Using FIFO scheduling algorithm.
(pid=74649) Resources requested: 18.0/72 CPUs, 0/0 GPUs, 0.0/339.53 GiB heap, 0.0/149.51 GiB objects
(pid=74649) Result logdir: /lus/scratch/arigazzi/ray_local/PPO
(pid=74649) Number of trials: 50/50 (44 PENDING, 6 RUNNING)
(pid=74649) 
(pid=74649) 
(pid=74649) == Status ==
(pid=74649) Memory usage on this node: 6.6/125.8 GiB
(pid=74649) Using FIFO scheduling algorithm.
(pid=74649) Resources requested: 72.0/72 CPUs, 0/0 GPUs, 0.0/339.53 GiB heap, 0.0/149.51 GiB objects
(pid=74649) Result logdir: /lus/scratch/arigazzi/ray_local/PPO
(pid=74649) Number of trials: 50/50 (26 PENDING, 24 RUNNING)
(pid=74649) 
(pid=74649) 


(pid=61437) 2021-05-24 07:56:07,687	INFO trainer.py:694 -- Current log_level is ERROR. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=61519) 2021-05-24 07:56:07,691	INFO trainer.py:694 -- Current log_level is ERROR. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=61507) 2021-05-24 07:56:07,685	INFO trainer.py:694 -- Current log_level is ERROR. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=61536) 2021-05-24 07:56:07,678	INFO trainer.py:694 -- Current log_level is ERROR. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=61545) 2021-05-24 07:56:07,683	INFO trainer.py:694 -- Current log_level is ERROR. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=61571) 2021-05-24 07:56:07,680	INFO trainer.py:694 -- Current log_level is ERROR. For more information, set 'log_level': 'INFO' / 'DEBU

(pid=74649) == Status ==
(pid=74649) Memory usage on this node: 6.6/125.8 GiB
(pid=74649) Using FIFO scheduling algorithm.
(pid=74649) Resources requested: 69.0/72 CPUs, 0/0 GPUs, 0.0/339.53 GiB heap, 0.0/149.51 GiB objects
(pid=74649) Result logdir: /lus/scratch/arigazzi/ray_local/PPO
(pid=74649) Number of trials: 50/50 (26 PENDING, 23 RUNNING, 1 TERMINATED)
(pid=74649) 
(pid=74649) 
(pid=74649) == Status ==
(pid=74649) Memory usage on this node: 6.5/125.8 GiB
(pid=74649) Using FIFO scheduling algorithm.
(pid=74649) Resources requested: 69.0/72 CPUs, 0/0 GPUs, 0.0/339.53 GiB heap, 0.0/149.51 GiB objects
(pid=74649) Result logdir: /lus/scratch/arigazzi/ray_local/PPO
(pid=74649) Number of trials: 50/50 (25 PENDING, 23 RUNNING, 2 TERMINATED)
(pid=74649) 
(pid=74649) 


...

(pid=3831) 2021-05-24 08:04:16,991	INFO trainer.py:694 -- Current log_level is ERROR. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


(pid=74649) == Status ==
(pid=74649) Memory usage on this node: 5.9/125.8 GiB
(pid=74649) Using FIFO scheduling algorithm.
(pid=74649) Resources requested: 57.0/72 CPUs, 0/0 GPUs, 0.0/339.53 GiB heap, 0.0/149.51 GiB objects
(pid=74649) Result logdir: /lus/scratch/arigazzi/ray_local/PPO
(pid=74649) Number of trials: 50/50 (19 RUNNING, 31 TERMINATED)
(pid=74649) 
(pid=74649) 


(pid=3831) 2021-05-24 08:04:29,025	INFO trainable.py:101 -- Trainable.setup took 12.034 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


(pid=74649) == Status ==
(pid=74649) Memory usage on this node: 5.9/125.8 GiB
(pid=74649) Using FIFO scheduling algorithm.
(pid=74649) Resources requested: 54.0/72 CPUs, 0/0 GPUs, 0.0/339.53 GiB heap, 0.0/149.51 GiB objects
(pid=74649) Result logdir: /lus/scratch/arigazzi/ray_local/PPO
(pid=74649) Number of trials: 50/50 (18 RUNNING, 32 TERMINATED)
(pid=74649) 
(pid=74649) 
(pid=74649) == Status ==
(pid=74649) Memory usage on this node: 5.4/125.8 GiB
(pid=74649) Using FIFO scheduling algorithm.
(pid=74649) Resources requested: 33.0/72 CPUs, 0/0 GPUs, 0.0/339.53 GiB heap, 0.0/149.51 GiB objects
(pid=74649) Result logdir: /lus/scratch/arigazzi/ray_local/PPO
(pid=74649) Number of trials: 50/50 (11 RUNNING, 39 TERMINATED)
(pid=74649) 
(pid=74649) 
(pid=74649) == Status ==
(pid=74649) Memory usage on this node: 4.6/125.8 GiB
(pid=74649) Using FIFO scheduling algorithm.
(pid=74649) Resources requested: 21.0/72 CPUs, 0/0 GPUs, 0.0/339.53 GiB heap, 0.0/149.51 GiB objects
(pid=74649) Result log

(pid=3831) 2021-05-24 08:06:08,388	ERROR worker.py:382 -- SystemExit was raised from the worker
(pid=3831) Traceback (most recent call last):
(pid=3831)   File "python/ray/_raylet.pyx", line 495, in ray._raylet.execute_task
(pid=3831)   File "python/ray/_raylet.pyx", line 505, in ray._raylet.execute_task
(pid=3831)   File "python/ray/_raylet.pyx", line 449, in ray._raylet.execute_task.function_executor
(pid=3831)   File "/lus/scratch/arigazzi/anaconda3/envs/smartsim/lib/python3.8/site-packages/ray/_private/function_manager.py", line 556, in actor_method_executor
(pid=3831)     return method(__ray_actor, *args, **kwargs)
(pid=3831)   File "/lus/scratch/arigazzi/anaconda3/envs/smartsim/lib/python3.8/site-packages/ray/actor.py", line 1001, in __ray_terminate__
(pid=3831)     ray.actor.exit_actor()
(pid=3831)   File "/lus/scratch/arigazzi/anaconda3/envs/smartsim/lib/python3.8/site-packages/ray/actor.py", line 1077, in exit_actor
(pid=3831)     raise exit
(pid=3831) SystemExit: 0
(pid=3831)

## 3. Stop cluster and release allocation

In [5]:
if alloc:
    slurm.release_allocation(alloc)

In [9]:
exp.stop(cluster)

06:39:40 swan SmartSim[1694] INFO Stopping model workers with job name workers-CBLEF8M2VDYD
06:39:40 swan SmartSim[1694] DEBUG Process terminated with kill 2252
06:39:40 swan SmartSim[1694] INFO Stopping model head with job name head-CBLEF2ZK1XJI
06:39:40 swan SmartSim[1694] DEBUG Process terminated with kill 1992
06:39:42 swan SmartSim[1694] DEBUG Sleeping, no jobs to monitor


In [6]:

tune.run(
    "PPO",
    stop={"episode_reward_max": 200},
    config={
        "framework": "torch",
        "env": "CartPole-v0",
    #    "num_gpus": 0,
        "lr": tune.grid_search(np.linspace (0.001, 0.01, 50).tolist()),
    #    "log_level": "ERROR",
    #    "num_cpus_per_worker": 1,
    },
    #local_dir="/lus/scratch/arigazzi/ray_local/",
    #verbose=1,
    #fail_fast=True,
    #log_to_file=True,
    reporter = JupyterNotebookReporter(True),
)

(pid=15462) == Status ==
(pid=15462) Memory usage on this node: 4.8/187.6 GiB
(pid=15462) Using FIFO scheduling algorithm.
(pid=15462) Resources requested: 0/72 CPUs, 0/0 GPUs, 0.0/512.51 GiB heap, 0.0/223.64 GiB objects
(pid=15462) Result logdir: /home/users/arigazzi/ray_results/PPO
(pid=15462) Number of trials: 50/50 (50 PENDING)
(pid=15462) 
(pid=15462) 
(pid=15462) == Status ==
(pid=15462) Memory usage on this node: 5.2/187.6 GiB
(pid=15462) Using FIFO scheduling algorithm.
(pid=15462) Resources requested: 0/72 CPUs, 0/0 GPUs, 0.0/512.51 GiB heap, 0.0/223.64 GiB objects
(pid=15462) Result logdir: /home/users/arigazzi/ray_results/PPO
(pid=15462) Number of trials: 50/50 (50 PENDING)
(pid=15462) 
(pid=15462) 
(pid=15462) == Status ==
(pid=15462) Memory usage on this node: 5.6/187.6 GiB
(pid=15462) Using FIFO scheduling algorithm.
(pid=15462) Resources requested: 0/72 CPUs, 0/0 GPUs, 0.0/512.51 GiB heap, 0.0/223.64 GiB objects
(pid=15462) Result logdir: /home/users/arigazzi/ray_results

(pid=15462) 2021-05-24 05:48:05,917	WARNING util.py:161 -- The `start_trial` operation took 0.504 s, which may be a performance bottleneck.


(pid=15462) == Status ==
(pid=15462) Memory usage on this node: 5.9/187.6 GiB
(pid=15462) Using FIFO scheduling algorithm.
(pid=15462) Resources requested: 9.0/72 CPUs, 0/0 GPUs, 0.0/512.51 GiB heap, 0.0/223.64 GiB objects
(pid=15462) Result logdir: /home/users/arigazzi/ray_results/PPO
(pid=15462) Number of trials: 50/50 (47 PENDING, 3 RUNNING)
(pid=15462) 
(pid=15462) 


(pid=15462) 2021-05-24 05:48:06,478	WARNING util.py:161 -- The `start_trial` operation took 0.522 s, which may be a performance bottleneck.
(pid=24081) 2021-05-24 05:48:14,554	INFO trainer.py:694 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=24080) 2021-05-24 05:48:14,633	INFO trainer.py:694 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=25721) 2021-05-24 05:48:14,861	INFO trainer.py:694 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=25724) 2021-05-24 05:48:14,957	INFO trainer.py:694 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=24494) 2021-05-24 05:48:15,035	INFO trainer.py:694 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=

(pid=15462) == Status ==
(pid=15462) Memory usage on this node: 7.4/187.6 GiB
(pid=15462) Using FIFO scheduling algorithm.
(pid=15462) Resources requested: 72.0/72 CPUs, 0/0 GPUs, 0.0/512.51 GiB heap, 0.0/223.64 GiB objects
(pid=15462) Result logdir: /home/users/arigazzi/ray_results/PPO
(pid=15462) Number of trials: 50/50 (26 PENDING, 24 RUNNING)
(pid=15462) 
(pid=15462) 
(pid=15462) == Status ==
(pid=15462) Memory usage on this node: 7.4/187.6 GiB
(pid=15462) Using FIFO scheduling algorithm.
(pid=15462) Resources requested: 72.0/72 CPUs, 0/0 GPUs, 0.0/512.51 GiB heap, 0.0/223.64 GiB objects
(pid=15462) Result logdir: /home/users/arigazzi/ray_results/PPO
(pid=15462) Number of trials: 50/50 (26 PENDING, 24 RUNNING)
(pid=15462) 
(pid=15462) 
(pid=15462) == Status ==
(pid=15462) Memory usage on this node: 7.4/187.6 GiB
(pid=15462) Using FIFO scheduling algorithm.
(pid=15462) Resources requested: 72.0/72 CPUs, 0/0 GPUs, 0.0/512.51 GiB heap, 0.0/223.64 GiB objects
(pid=15462) Result logdir:

...

(pid=31284) 2021-05-24 05:55:45,750	INFO trainable.py:101 -- Trainable.setup took 13.816 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
(pid=31291) 2021-05-24 05:55:45,800	INFO trainable.py:101 -- Trainable.setup took 13.793 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


(pid=15462) == Status ==
(pid=15462) Memory usage on this node: 7.0/187.6 GiB
(pid=15462) Using FIFO scheduling algorithm.
(pid=15462) Resources requested: 60.0/72 CPUs, 0/0 GPUs, 0.0/512.51 GiB heap, 0.0/223.64 GiB objects
(pid=15462) Result logdir: /home/users/arigazzi/ray_results/PPO
(pid=15462) Number of trials: 50/50 (20 RUNNING, 30 TERMINATED)
(pid=15462) 
(pid=15462) 
(pid=15462) == Status ==
(pid=15462) Memory usage on this node: 7.0/187.6 GiB
(pid=15462) Using FIFO scheduling algorithm.
(pid=15462) Resources requested: 60.0/72 CPUs, 0/0 GPUs, 0.0/512.51 GiB heap, 0.0/223.64 GiB objects
(pid=15462) Result logdir: /home/users/arigazzi/ray_results/PPO
(pid=15462) Number of trials: 50/50 (20 RUNNING, 30 TERMINATED)
(pid=15462) 
(pid=15462) 
(pid=15462) == Status ==
(pid=15462) Memory usage on this node: 7.0/187.6 GiB
(pid=15462) Using FIFO scheduling algorithm.
(pid=15462) Resources requested: 60.0/72 CPUs, 0/0 GPUs, 0.0/512.51 GiB heap, 0.0/223.64 GiB objects
(pid=15462) Result l

(pid=31284) 2021-05-24 05:56:06,436	WARNING deprecation.py:33 -- DeprecationWarning: `SampleBatch.data[..]` has been deprecated. Use `SampleBatch[..]` instead. This will raise an error in the future!
(pid=31291) 2021-05-24 05:56:06,564	WARNING deprecation.py:33 -- DeprecationWarning: `SampleBatch.data[..]` has been deprecated. Use `SampleBatch[..]` instead. This will raise an error in the future!


(pid=15462) == Status ==
(pid=15462) Memory usage on this node: 7.0/187.6 GiB
(pid=15462) Using FIFO scheduling algorithm.
(pid=15462) Resources requested: 57.0/72 CPUs, 0/0 GPUs, 0.0/512.51 GiB heap, 0.0/223.64 GiB objects
(pid=15462) Result logdir: /home/users/arigazzi/ray_results/PPO
(pid=15462) Number of trials: 50/50 (19 RUNNING, 31 TERMINATED)
(pid=15462) 
(pid=15462) 
(pid=15462) == Status ==
(pid=15462) Memory usage on this node: 7.0/187.6 GiB
(pid=15462) Using FIFO scheduling algorithm.
(pid=15462) Resources requested: 36.0/72 CPUs, 0/0 GPUs, 0.0/512.51 GiB heap, 0.0/223.64 GiB objects
(pid=15462) Result logdir: /home/users/arigazzi/ray_results/PPO
(pid=15462) Number of trials: 50/50 (12 RUNNING, 38 TERMINATED)
(pid=15462) 
(pid=15462) 
(pid=15462) == Status ==
(pid=15462) Memory usage on this node: 7.0/187.6 GiB
(pid=15462) Using FIFO scheduling algorithm.
(pid=15462) Resources requested: 21.0/72 CPUs, 0/0 GPUs, 0.0/512.51 GiB heap, 0.0/223.64 GiB objects
(pid=15462) Result l

(pid=38507) 2021-05-24 05:57:29,347	ERROR worker.py:382 -- SystemExit was raised from the worker
(pid=38507) Traceback (most recent call last):
(pid=38507)   File "python/ray/_raylet.pyx", line 595, in ray._raylet.task_execution_handler
(pid=38507)   File "/lus/scratch/arigazzi/anaconda3/envs/smartsim/lib/python3.8/site-packages/ray/_private/client_mode_hook.py", line 18, in _disable_client_hook
(pid=38507)     def _disable_client_hook():
(pid=38507)   File "/lus/scratch/arigazzi/anaconda3/envs/smartsim/lib/python3.8/site-packages/ray/worker.py", line 379, in sigterm_handler
(pid=38507)     sys.exit(1)
(pid=38507) SystemExit: 1
(pid=38508) 2021-05-24 05:57:29,347	ERROR worker.py:382 -- SystemExit was raised from the worker
(pid=38508) Traceback (most recent call last):
(pid=38508)   File "python/ray/_raylet.pyx", line 595, in ray._raylet.task_execution_handler
(pid=38508)   File "/lus/scratch/arigazzi/anaconda3/envs/smartsim/lib/python3.8/site-packages/ray/_private/client_mode_hook.py"

In [8]:
analysis = tune.run(
    "PPO",
    stop={"episode_reward_max": 200},
    config={
        "framework": "torch",
        "env": "CartPole-v0",
        "lr": tune.grid_search([0.001, 0.002, 0.003, 0.004, 0.005]),
    },
    progress_reporter = JupyterNotebookReporter(True)
)

<IPython.core.display.HTML object>
<IPython.core.display.HTML object>


(pid=31372) 2021-05-24 05:58:41,433	INFO trainer.py:694 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=32926) 2021-05-24 05:58:42,009	INFO trainer.py:694 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=31372) 2021-05-24 05:58:48,558	WARNING deprecation.py:33 -- DeprecationWarning: `SampleBatch.data[..]` has been deprecated. Use `SampleBatch[..]` instead. This will raise an error in the future!
(pid=32926) 2021-05-24 05:58:49,178	WARNING deprecation.py:33 -- DeprecationWarning: `SampleBatch.data[..]` has been deprecated. Use `SampleBatch[..]` instead. This will raise an error in the future!


<IPython.core.display.HTML object>


(pid=40976) 2021-05-24 05:58:55,399	INFO trainer.py:694 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=31101) 2021-05-24 05:58:55,836	INFO trainer.py:694 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=31167) 2021-05-24 05:58:55,834	INFO trainer.py:694 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


<IPython.core.display.HTML object>


(pid=40976) 2021-05-24 05:59:03,475	WARNING deprecation.py:33 -- DeprecationWarning: `SampleBatch.data[..]` has been deprecated. Use `SampleBatch[..]` instead. This will raise an error in the future!


<IPython.core.display.HTML object>


(pid=31167) 2021-05-24 05:59:09,698	WARNING deprecation.py:33 -- DeprecationWarning: `SampleBatch.data[..]` has been deprecated. Use `SampleBatch[..]` instead. This will raise an error in the future!
(pid=31101) 2021-05-24 05:59:09,809	WARNING deprecation.py:33 -- DeprecationWarning: `SampleBatch.data[..]` has been deprecated. Use `SampleBatch[..]` instead. This will raise an error in the future!


<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>


(pid=31125) 2021-05-24 05:59:50,120	ERROR worker.py:382 -- SystemExit was raised from the worker
(pid=31125) Traceback (most recent call last):
(pid=31125)   File "python/ray/_raylet.pyx", line 495, in ray._raylet.execute_task
(pid=31125)   File "python/ray/_raylet.pyx", line 505, in ray._raylet.execute_task
(pid=31125)   File "python/ray/_raylet.pyx", line 449, in ray._raylet.execute_task.function_executor
(pid=31125)   File "/lus/scratch/arigazzi/anaconda3/envs/smartsim/lib/python3.8/site-packages/ray/_private/function_manager.py", line 556, in actor_method_executor
(pid=31125)     return method(__ray_actor, *args, **kwargs)
(pid=31125)   File "/lus/scratch/arigazzi/anaconda3/envs/smartsim/lib/python3.8/site-packages/ray/actor.py", line 1001, in __ray_terminate__
(pid=31125)     ray.actor.exit_actor()
(pid=31125)   File "/lus/scratch/arigazzi/anaconda3/envs/smartsim/lib/python3.8/site-packages/ray/actor.py", line 1077, in exit_actor
(pid=31125)     raise exit
(pid=31125) SystemExit: 